# Distances in the C-space
In this seminar we will practice some of the concepts discussed in L03

## Distance for orientations
Angles are the way to express orientations in 2D, that means, for any object that has dimensions, there is going to be an orientation variable that expresses their state.

We will start with angles, since this is also one of the easiest manifolds to study. We started presenting angles as elements of the 1-spherical group $\mathbb{S} = \{ (x,y) \in \mathbb{R}^2 | x^2+y^2=1\}$.

Our first task is to play around with distances here, below is provided some code for visualization.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def plot_segment(s, color_):
    """
    Plots a segment in current figures
    """
    plt.plot(s[:,0], s[:,1] , linewidth=2, color=color_)
    plt.plot(s[1,0], s[1,1], marker='o', color=color_)
    plt.plot(s[0,0], s[0,1], marker='o', color=color_)

def plot_angles(angle1, angle2):
    """
    Plots a pair of angles in the S group
    """
    plt.figure(figsize=(9, 9))
    n = 50
    circle = np.zeros((2, n))
    for i in range(n):
        theta = i / (n - 1) * 2 * np.pi
        x, y = np.cos(theta), np.sin(theta)
        circle[:, i] = [x, y]
    plt.plot(circle[0,:], circle[1,:])
    segment1 = np.array([[0,0],
                         [np.cos(angle1), np.sin(angle1)]])
    color = np.array([1,0,0])
    plot_segment(segment1, color)
    segment2 = np.array([[0,0],
                         [np.cos(angle2), np.sin(angle2)]])
    color = np.array([0,1,0])
    plot_segment(segment2, color)
    plt.show()

def wrap_angle(angle):
    """
    Wraps the given angle to the range [-pi, +pi].

    :param angle: The angle (in rad) to wrap (can be unbounded).
    :return: The wrapped angle (guaranteed to in [-pi, +pi]).
    """

    pi2 = 2 * np.pi

    while angle < -np.pi:
        angle += pi2

    while angle >= np.pi:
        angle -= pi2

    return angle

### Task 1: Plot some random pair of configurations and calculate the raw distance between angles and proper angle distance as decribed in class

Check for distances comparing both angles and visualize its value as elements of $\mathbb{S}$. Interesting points around 3, 6, -6, etc

In [ ]:
angle1, angle2 = 0.0, 0.1  # 1: red, 2: green
plot_angles(angle1, angle2)
standart_dist = ...
correct_dist = ...
print('standard distance = ', standart_dist)
print('Angle distance = ', correct_dist)

### Task 2: fix one of the angles and range the values of the second variable
Plot the resultant figure, for distances and angle distances for $\theta_1 = 0$

$f(\theta_2) = d(\theta_1,\theta_2), \quad \theta_2 \in [-10,10]$

In [ ]:
# for angle in N:
#    standart_dist[angle] = ...
#    correct_dist[angle] = ...
# plt.plot ...

### (optional topic) There is an alternative to describe angles: $SO(2)$

$SO(2)$ are the group of rotations in 2D. More formally: $SO(2) = \{R \in \mathbb{R}^{2\times 2} | R\cdot R^{\top} = I, \text{det}(R) = 1 \}$.

For this, we know that only a single variable is required to determine all possible rotation matrices:

$$
R(\theta) = \begin{bmatrix}
  \cos (\theta) & - \sin (\theta )\\
  \sin ( \theta)  &  \cos ( \theta)
\end{bmatrix}
$$

Distances can be defined in terms of rotation matrices, and the result is exactly the same as wraping. So, for 2D, it does not make sense but for 3D it could be a very useful tool for representing rotations and measuring distances:

$$d(\theta_1,\theta_2) = d(R_1,R_2) = || \text{Log}(R_1 \cdot R_2^{\top})||$$

where the $\text{Log}()$ function can be understand as an inverse mapping from the rotation matrix to a single variable, the angle.

### Task 3: Distance of 2D poses
for this task, we will consider 2d poses as a position and an orientation $q = [x,y,\theta]^{\top} \in \mathbb{R}^2\times \mathbb{S}$.

The first task is to construct a distance function for any two configurations. Use the visualization tools for ploting.

<!-- For a more comprehensive explanation on 2D poses, you can take a look at the lecture 5 in the course Perception in Robotics, class notes and the seminar work. -->

In [ ]:
def plot_2d_poses(q1,q2):
    """given two configurations q1, q2, the function plots the poses of 2D objects"""
    triangle = np.array([[-1,-1],
                         [-1,1],
                         [1.5,0]])
    plt.figure(figsize=(9, 9))
    color = np.array([1,0,0])
    # Transform first object to the world coordinates
    triangle1 = np.zeros((3,2))
    for i in range(3):
        triangle1[i,:] = transform_point(q1,triangle[i,:])
    plot_segment(triangle1[[0,1],:], color)
    plot_segment(triangle1[[0,2],:], color)
    plot_segment(triangle1[[1,2],:], color)
    plt.plot(q1[0], q1[1], markersize=15, marker='*', color=color)
    
    # Transform second object to the world coordinates
    triangle2 = np.zeros((3,2))
    for i in range(3):
        triangle2[i,:] = transform_point(q2,triangle[i,:])
    color = np.array([0,1,0])
    plot_segment(triangle2[[0,1],:], color)
    plot_segment(triangle2[[0,2],:], color)
    plot_segment(triangle2[[1,2],:], color)
    plt.plot(q2[0], q2[1], markersize=15, marker='*', color=color)
    #plt.plot(triangle2[0,0], triangle2[0,1], markersize=15, marker='*', color=color)
    plt.axis('equal')
    plt.show()
    

def c(t):
    return np.cos(t)

def s(t):
    return np.sin(t)

def SE2(q):
    """Returns the SE2  transformation matrix consisting of a translation and rotation in 2-D"""
    x, y, t = q
    T = np.array([
        [c(t), -s(t), x],
        [s(t), c(t), y],
        [0, 0, 1]])
    return T

def transform_point(q,p):
    """Given a pose q = [x,y,theta] and a point p = [x,y], it transforms 
    according to p' = R(theta)p + t. This is a rigid body transformation in 2D"""
    T = SE2(q)
    p_homogeneous = np.array([p[0],p[1],1])
    p_new = T @ p_homogeneous
    return p_new[:2]

### Task 4: Play around with different configuration and propose a distance metric
There is not right answer here, define a new metric for measuring distances between the 2 objects

In [ ]:
q1 = np.array([0,0,0]) # red
q2 = np.array([0,1,0]) # green
plot_2d_poses(q1,q2)

In [ ]:
# def get_distance(q1,q2):
# return ...

### task 5: 2R manipulator
The 2R manipulator consist of 2 revolute joints. Here, the configuration space is $q = [\theta_1, \theta_2] \in \mathbb{S}\times \mathbb{S}$.

Use the utilities provided below to visualize different configurations while defining different distance functions:

 * Distance between joint values
 * Weighted distance between joint values
 * Distance of the end effector in workspace
 * Other alternatives

In [ ]:
def plot_2R(q, color):
    """Plots the 2R manipulator, a kinematic chain of 2 bars of length 1"""
    # segment 1:
    seg = np.zeros((3,2))
    a1,a2 = q
    T1 = SE2(np.array([0,0,a1])) # this is the first joint, a simple rotation
    T2 = SE2(np.array([1,0,a2])) # the second joint, it is a bar of d =1, plus a rotation for the second joint
    T3 = SE2(np.array([1,0,0]))  # this is simply to express the lenght of the second bar, no rotation required
    p = T1 @ T2 @ np.array([0,0,1])
    seg[1,:] = p[:2]
    p = T1 @ T2 @ T3 @ np.array([0,0,1])
    seg[2,:] = p[:2]
    plot_segment(seg[[0,1],:], color)
    plot_segment(seg[[1,2],:], color)
    plt.axis('equal')
    return seg[1,:], seg[2,:]

def plot_pair_2R(q1,q2):
    """Plot a pair of 2R configurations"""
    c = np.array([1,0,0])
    plot_2R(q1,c)
    c = np.array([0,1,0])
    plot_2R(q2,c)
    plt.axis('equal')
    plt.show()

In [ ]:
color = np.array([1,0,0])
p1,p2 = plot_2R(np.array([0.0,0.0]),color)
print(p1,p2)

In [ ]:
q1 = np.array([0,0]) # red
q2 = np.array([0.2,- np.pi+0.1]) # green
plot_pair_2R(q1,q2)